['google_robot_pick_coke_can', 'google_robot_pick_horizontal_coke_can', 'google_robot_pick_vertical_coke_can', 'google_robot_pick_standing_coke_can', 'google_robot_pick_object', 'google_robot_move_near_v0', 'google_robot_move_near_v1', 'google_robot_move_near', 'google_robot_open_drawer', 'google_robot_open_top_drawer', 'google_robot_open_middle_drawer', 'google_robot_open_bottom_drawer', 'google_robot_close_drawer', 'google_robot_close_top_drawer', 'google_robot_close_middle_drawer', 'google_robot_close_bottom_drawer', 'google_robot_place_in_closed_drawer', 'google_robot_place_in_closed_top_drawer', 'google_robot_place_in_closed_middle_drawer', 'google_robot_place_in_closed_bottom_drawer', 'google_robot_place_apple_in_closed_top_drawer', 'widowx_spoon_on_towel', 'widowx_carrot_on_plate', 'widowx_stack_cube', 'widowx_put_eggplant_in_basket']

google_robot_pick_coke_can
google_robot_open_drawer
widowx_stack_cube

In [6]:
import simpler_env
from simpler_env.utils.env.observation_utils import get_image_from_maniskill2_obs_dict
import numpy as np
import mediapy as media
from pathlib import Path
import requests
import cv2
import io


In [9]:
response = requests.post(
        "http://localhost:8003/reset",
        data={ "task_description": "Bro, open middle drawer" }
    )
result = response.json()
result

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [5]:

env = simpler_env.make("widowx_put_eggplant_in_basket")
obs, _  = env.reset()
instr   = env.get_language_instruction()

success = truncated = False
frames  = []

while not (success or truncated):
    img = get_image_from_maniskill2_obs_dict(env, obs)   # H×W×3, uint8
    frames.append(img)
   # cv2.imwrite('output_image.jpg', img)

    success, jpeg_bytes = cv2.imencode('.jpg', img)
    if not success:
        raise RuntimeError("Failed to encode image")
    
    response = requests.post(
        "http://localhost:8003/step",
        files={
            # The key "image" should match the parameter name in your API
            "file": ("image.jpg", jpeg_bytes.tobytes(), "image/jpeg"),
        },
        data={
            "task_description": instr,
        }
    )
    # env.get_language_instruction()
    result = response.json()
    act = result["action"]
    print(act)

    vec   = np.concatenate([act["world_vector"],
                            act["rot_axangle"],
                            act["gripper"]])
    obs, _, success, truncated, info = env.step(vec)

tag = "success" if success else "fail"
# mediapy.write_video expects (T, H, W, 3)
media.write_video("./tmp2.mp4", np.stack(frames), fps=10, codec="h264")
print(tag)

[2025-04-25 01:52:45.376] [svulkan2] [warning] A second renderer will share the same internal context with the first one. Arguments passed to constructor will be ignored.


ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [6]:
media.write_video("./tmp2.mp4", np.stack(frames), fps=10, codec="h264")


In [1]:
# ------------------------------------------------------------------
TASK        = "google_robot_open_drawer"
N_EPISODES  = 30          # how many roll-outs you want
FPS         = 10          # playback speed for the saved videos
OUTDIR      = Path("videos/spatial_vla")
OUTDIR.mkdir(exist_ok=True)
# ------------------------------------------------------------------

env = simpler_env.make(TASK)

# policy = OpenVLAInference(
#     saved_model_path="/root/AK103/tmp/AK103/SimplerEnv-OpenVLA/checkpoints/openvla-7b",
#     policy_setup="google_robot",
# )

policy = SpatialVLAInference(
    saved_model_path="/root/AK103/tmp/AK103/SimplerEnv-OpenVLA/checkpoints/spatialvla-4b",
    policy_setup="google_robot",
)

success_counter = 0

for ep in range(N_EPISODES):
    obs, _  = env.reset()
    instr   = env.get_language_instruction()
    policy.reset(instr)

    success = truncated = False
    frames  = []                             # ← new: gather video frames

    while not (success or truncated):
        img = get_image_from_maniskill2_obs_dict(env, obs)   # H×W×3, uint8
        frames.append(img)

        _, act = policy.step(img, instr)
        vec   = np.concatenate([act["world_vector"],
                                act["rot_axangle"],
                                act["gripper"]])
        obs, _, success, truncated, info = env.step(vec)

    # ------------------------------------------------------------------
    # Save the video of this episode
    # ------------------------------------------------------------------
    tag         = "success" if success else "fail"
    video_path  = OUTDIR / f"episode_{ep:02d}_{tag}.mp4"
    # mediapy.write_video expects (T, H, W, 3)
    media.write_video(video_path, np.stack(frames), fps=FPS, codec="h264")

    success_counter += int(success)
    print(f"Episode {ep:02d}  ⇒  {'✓' if success else '✗'}   | saved → {video_path}")

# ------------------------------------------------------------------
print(f"\nAverage success over {N_EPISODES} runs: "
      f"{success_counter / N_EPISODES:.3f} "
      f"({success_counter}/{N_EPISODES})")

['google_robot_pick_coke_can', 'google_robot_pick_horizontal_coke_can', 'google_robot_pick_vertical_coke_can', 'google_robot_pick_standing_coke_can', 'google_robot_pick_object', 'google_robot_move_near_v0', 'google_robot_move_near_v1', 'google_robot_move_near', 'google_robot_open_drawer', 'google_robot_open_top_drawer', 'google_robot_open_middle_drawer', 'google_robot_open_bottom_drawer', 'google_robot_close_drawer', 'google_robot_close_top_drawer', 'google_robot_close_middle_drawer', 'google_robot_close_bottom_drawer', 'google_robot_place_in_closed_drawer', 'google_robot_place_in_closed_top_drawer', 'google_robot_place_in_closed_middle_drawer', 'google_robot_place_in_closed_bottom_drawer', 'google_robot_place_apple_in_closed_top_drawer', 'widowx_spoon_on_towel', 'widowx_carrot_on_plate', 'widowx_stack_cube', 'widowx_put_eggplant_in_basket']
